### Instalación de las dependencias

In [ ]:
!pip install PyPDF2
!pip install pdf2image
!pip install pytesseract
!pip install spacy
!pip install gensim
!python -m spacy download es_core_news_sm
!pip install textblob

### Instalaciones de Librerías

In [ ]:
import os
import re
import spacy
import nltk
import PyPDF2
import pandas as pd
import pytesseract
import matplotlib.pyplot as plt
from textblob import TextBlob
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from pdf2image import convert_from_path
from pytesseract import Output
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

In [13]:
!apt-get update
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr tesseract-ocr-spa

### Instalaciones de Recursos

In [ ]:
nltk.download('wordnet')
nlp = spacy.load("es_core_news_sm")

##LECTURA DE PDFs

In [ ]:
import os
import PyPDF2
from pdf2image import convert_from_path
import pytesseract
from pytesseract import Output

# Ruta del directorio donde están los PDFs
ruta_directorio = "/content/resoluciones/"

# Función para extraer texto de PDFs normales
def extraer_texto_pdf(ruta_archivo):
    texto = ""
    try:
        with open(ruta_archivo, 'rb') as archivo_pdf:
            lector_pdf = PyPDF2.PdfReader(archivo_pdf)
            for pagina in range(len(lector_pdf.pages)):
                texto += lector_pdf.pages[pagina].extract_text() or ""
    except PyPDF2.errors.PdfReadError as e:
        print(f"Error al leer el archivo {ruta_archivo}: {e}")
    except Exception as e:
        print(f"Ocurrió un error al procesar {ruta_archivo}: {e}")
    return texto

# Función para extraer texto de PDFs que contienen imágenes usando OCR
def extraer_texto_ocr(ruta_archivo):
    texto = ""
    try:
        # Convertir las páginas del PDF a imágenes
        paginas_imagen = convert_from_path(ruta_archivo)
        for pagina_imagen in paginas_imagen:
            # Aplicar OCR para extraer el texto de la imagen
            texto += pytesseract.image_to_string(pagina_imagen, lang='spa')  # OCR en español
    except Exception as e:
   


##LIMPIEZA DE TEXTO


In [ ]:
# Rutas de Poppler y Tesseract
ruta_poppler = "/usr/bin"
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"  # Configurar Tesseract

# Función para extraer texto de PDFs normales
def extraer_texto_pdf(ruta_archivo):
    texto = ""
    try:
        with open(ruta_archivo, 'rb') as archivo_pdf:
            lector_pdf = PyPDF2.PdfReader(archivo_pdf)
            for pagina in range(len(lector_pdf.pages)):
                texto += lector_pdf.pages[pagina].extract_text() or ""
    except PyPDF2.errors.PdfReadError as e:
        print(f"Error al leer el archivo {ruta_archivo}: {e}")
    except Exception as e:
        print(f"Ocurrió un error al procesar {ruta_archivo}: {e}")
    return texto

# Función para extraer texto de PDFs con imágenes usando OCR
def extraer_texto_ocr(ruta_archivo):
    texto = ""
    try:
        paginas_imagen = convert_from_path(ruta_archivo, poppler_path=ruta_poppler)
        for pagina_imagen in paginas_imagen:
            texto += pytesseract.image_to_string(pagina_imagen, lang='spa')
    except Exception as e:
        print(f"Error al aplicar OCR a {ruta_archivo}: {e}")
    return texto

# Función para procesar PDFs (detectar si tienen texto o son imágenes)
def procesar_pdf(ruta_archivo):
    try:
        texto = extraer_texto_pdf(ruta_archivo)
        if not texto.strip():
            print(f"Aplicando OCR al archivo {ruta_archivo}")
            texto = extraer_texto_ocr(ruta_archivo)
        return texto
    except Exception as e:
        print(f"Error general al procesar el archivo {ruta_archivo}: {e}")
        return ""

# Directorio con los PDFs
ruta_directorio = "/content/resoluciones/"
archivos_pdf = [f for f in os.listdir(ruta_directorio) if f.endswith('.pdf')]

# Leer y almacenar el contenido de todos los PDFs
resoluciones = []
for archivo in archivos_pdf:
    ruta_completa = os.path.join(ruta_directorio, archivo)
    texto = procesar_pdf(ruta_completa)
    if texto:
        resoluciones.append(texto)

# Mostrar el contenido de las primeras resoluciones extraídas
for i, texto in enumerate(resoluciones[:3]):
    print(f"Resolución {i + 1}:\n{texto[:500]}...\n")

### Limpieza del texto

In [ ]:
# Función para limpiar el texto
def limpiar_texto(texto):
    texto = texto.lower()  # Convertir a minúsculas
    texto = re.sub(r'\b(?!19|20)\d{1,3}\b', '', texto)  # Eliminar números irrelevantes
    texto = re.sub(r'https?://\S+|www\.\S+', ' ', texto)  # Eliminar URLs
    texto = re.sub(r'[^a-zA-Z0-9áéíóúüñ\s]', '', texto)  # Eliminar caracteres especiales
    texto = re.sub(r'\s+', ' ', texto).strip()  # Eliminar espacios extras
    return texto

# Aplicar la limpieza
resoluciones_limpias = [limpiar_texto(res) for res in resoluciones]

# Mostrar los primeros 1000 caracteres de las resoluciones limpias
texto_limpio = ' '.join(resoluciones_limpias)
print(texto_limpio[:1000])

##PREPROCESAMIENTO (Lematización y Stopwords)

In [ ]:
# Función para lematizar usando spaCy
def lematizar(texto):
    doc = nlp(texto)
    # Lematizar, eliminando puntuación y espacios
    lemas = [token.lemma_ for token in doc if not token.is_punct and not token.is_space]
    return ' '.join(lemas)

for word in ["dirección","general", "de", "cultura", "y", "educación", "la", "plata"]:
    nlp.vocab[word].is_stop = True

# Función para eliminar stopwords usando spaCy
def eliminar_stopwords(texto):
    doc = nlp(texto)
    # Eliminar stopwords, puntuación y espacios
    palabras_filtradas = [token.text for token in doc if not token.is_stop and not token.is_punct and not token.is_space]
    return ' '.join(palabras_filtradas)

# Aplicar limpieza, lematización y eliminación de stopwords
def procesar_resoluciones(res):
    res_procesada = lematizar(res)
    res_procesada = eliminar_stopwords(res_procesada)
    return res_procesada

# Procesar todas las resoluciones
resoluciones_procesadas = [procesar_resoluciones(res) for res in resoluciones_limpias]

# Mostrar el resultado de la primera resolución procesada
print(resoluciones_procesadas[0][:500])

### Análisis gramatical (POS tagging y NER)

In [ ]:
# Función para realizar POS tagging
def etiquetar_partes_discurso(texto):
    doc = nlp(texto)
    return [(token.text, token.pos_) for token in doc]

# Función para extraer entidades nombradas (NER)
def extraer_entidades(texto):
    doc = nlp(texto)
    entidades = [(ent.text, ent.label_) for ent in doc.ents]
    return entidades

# Extraer y mostrar POS y NER de las resoluciones procesadas
etiquetas = etiquetar_partes_discurso(' '.join(resoluciones_procesadas))
print("Etiquetas POS:", etiquetas)

entidades_extraidas = [extraer_entidades(res) for res in resoluciones_procesadas]
print("Entidades extraídas:", entidades_extraidas[0])


In [ ]:
cv = CountVectorizer().fit(resoluciones_procesadas)
print("Tamaño de vocabulario: {}".format(len(cv.vocabulary_)))
print("Vocabulario:\n{}".format(cv.get_feature_names_out()))

### ANÁLISIS con BoW

In [ ]:
# Inicializar el vectorizador BoW
vectorizer = CountVectorizer(max_features=1000, ngram_range=(3, 3))  # Ajustar ngram según sea necesario
X = vectorizer.fit_transform(resoluciones_procesadas)

# Convertir el resultado a un DataFrame
df_bow = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
print(df_bow.head())

In [ ]:
# Graficar las 20 palabras más frecuentes
palabras_importantes_bow = df_bow.sum().sort_values(ascending=False)
print(palabras_importantes_bow.head(20))

In [ ]:
# Definir un umbral para las palabras clave
umbral = 0.3  # Ajustar

# Crear un diccionario para almacenar los temas
temas = {}

# Agrupar palabras clave por temas
for i, row in df_bow.iterrows():
    palabras_importantes = row[row > umbral].index.tolist()  # Obtener palabras clave con conteo > umbral
    for palabra in palabras_importantes:
        if palabra not in temas:
            temas[palabra] = []  # Inicializa una lista para cada palabra clave
        temas[palabra].append(i)  # Agregar el índice de la resolución al tema correspondiente

print("Palabras clave por tema:")
for indice, palabras in temas.items():
    # Convertir cada elemento en 'palabras' a string
    palabras_str = [str(palabra) for palabra in palabras]
    print(f'Tema: {", ".join(palabras_str)}; Palabras clave: {indice}')

In [ ]:
# Crear un string para la nube de palabras, asegurando que las claves sean de tipo str
nube_texto = ' '.join([str(palabra) for palabra in temas.keys()])  # Unir las palabras clave

# Generar la nube de palabras
nube_palabras = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(nube_texto)

# Mostrar la nube de palabras
plt.figure(figsize=(10, 5))
plt.imshow(nube_palabras, interpolation='bilinear')
plt.axis('off')  # No mostrar los ejes
plt.title('Nube de Palabras de Palabras Clave por Tema (BoW)', fontsize=16)
plt.show()

##ANÁLISIS con TF-IDF

In [ ]:
# Inicializar el vectorizador TF-IDF
vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(3,3))
X_tfidf = vectorizer.fit_transform(resoluciones_procesadas)

# Convertir a DataFrame
df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
print(df_tfidf.head())


In [ ]:
# Mostrar las palabras más importantes
palabras_importantes_tfidf = df_tfidf.sum().sort_values(ascending=False)
print(palabras_importantes_tfidf.head(10))

In [ ]:
# Definir un umbral para las palabras clave
umbral = 0.3  # Ajusta según sea necesario

# Crear un diccionario para almacenar las palabras clave por tema
temas_tfidf = {}

# Agrupar palabras clave por temas
for i, row in df_tfidf.iterrows():
    palabras_importantes = row[row > umbral].index.tolist()  # Obtener palabras clave con puntuación TF-IDF > umbral
    temas_tfidf[i] = palabras_importantes  # Asociar el índice de la resolución con sus palabras clave

# Imprimir las palabras por tema
print("Palabras clave por tema:")
for indice, palabras in temas_tfidf.items():
    print(f'Tema {indice}: Palabras clave: {", ".join(palabras)}')


In [ ]:
# Crear un string para la nube de palabras
nube_texto_tfidf = ' '.join([palabra for palabras in temas_tfidf.values() for palabra in palabras])  # Unir todas las palabras clave

# Generar la nube de palabras
nube_palabras_tfidf = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(nube_texto_tfidf)

# Mostrar la nube de palabras
plt.figure(figsize=(10, 5))
plt.imshow(nube_palabras_tfidf, interpolation='bilinear')
plt.axis('off')  # No mostrar los ejes
plt.title('Nube de Palabras de Palabras Clave por Tema (TF-IDF)', fontsize=16)
plt.show()

### Clustering sobre BoW y TF-IDF

In [ ]:
from sklearn.metrics import silhouette_score

# Lista para almacenar los coeficientes de silhouette
silhouette_scores_bow = []
silhouette_scores_tfidf = []

# Probar diferentes números de clusters
for k in range(2, 11):  # El coeficiente silhouette no se define para k=1
    # K-means para BoW
    kmeans_bow = KMeans(n_clusters=k, random_state=42)
    clusters_bow = kmeans_bow.fit_predict(X)
    silhouette_scores_bow.append(silhouette_score(X, clusters_bow))

    # K-means para TF-IDF
    kmeans_tfidf = KMeans(n_clusters=k, random_state=42)
    clusters_tfidf = kmeans_tfidf.fit_predict(X_tfidf)
    silhouette_scores_tfidf.append(silhouette_score(X_tfidf, clusters_tfidf))

# Graficar el coeficiente de silhouette para BoW y TF-IDF
plt.figure(figsize=(12, 6))

# Silhouette score para BoW
plt.subplot(1, 2, 1)
plt.plot(range(2, 11), silhouette_scores_bow, marker='o', color='skyblue')
plt.title('Coeficiente Silhouette (BoW)')
plt.xlabel('Número de Clusters')
plt.ylabel('Silhouette Score')

# Silhouette score para TF-IDF
plt.subplot(1, 2, 2)
plt.plot(range(2, 11), silhouette_scores_tfidf, marker='o', color='salmon')
plt.title('Coeficiente Silhouette (TF-IDF)')
plt.xlabel('Número de Clusters')
plt.ylabel('Silhouette Score')

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.cluster import KMeans

# Número de clusters
n_clusters = 8  # Ajusta según sea necesario

# K-means para Bag of Words
kmeans_bow = KMeans(n_clusters=n_clusters, random_state=42)
clusters_bow = kmeans_bow.fit_predict(X)

# K-means para TF-IDF
kmeans_tfidf = KMeans(n_clusters=n_clusters, random_state=42)
clusters_tfidf = kmeans_tfidf.fit_predict(X_tfidf)


In [ ]:
# Contar la cantidad de documentos en cada cluster para BoW
conteo_clusters_bow = pd.Series(clusters_bow).value_counts().sort_index()

# Contar la cantidad de documentos en cada cluster para TF-IDF
conteo_clusters_tfidf = pd.Series(clusters_tfidf).value_counts().sort_index()

# Graficar los resultados
plt.figure(figsize=(12, 6))

# Gráfico para BoW
plt.subplot(1, 2, 1)
conteo_clusters_bow.plot(kind='bar', color='skyblue')
plt.title('Distribución de Documentos en Clusters (BoW)')
plt.xlabel('Cluster')
plt.ylabel('Cantidad de Documentos')
plt.xticks(rotation=0)

# Gráfico para TF-IDF
plt.subplot(1, 2, 2)
conteo_clusters_tfidf.plot(kind='bar', color='salmon')
plt.title('Distribución de Documentos en Clusters (TF-IDF)')
plt.xlabel('Cluster')
plt.ylabel('Cantidad de Documentos')
plt.xticks(rotation=0)

plt.tight_layout()
plt.show()
